### Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# ML Libraries
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error,root_mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

### Reading CSV Files

In [2]:
df = pd.read_csv('Copper_Cleaned.csv')

In [3]:
df.head()

,quantity tons,country,status,application,thickness,width,material_ref,selling_price,purchased_month,purchased_year,delivered_month,delivered_year,item_type_PL,item_type_S,item_type_W
0,5.64,28,1,1.71,0.69,992.83,4329,15.26,4.0,2021.0,7.0,2021.0,0,0,1
1,4.71,27,1,1.71,0.38,685.97,4243,15.83,4.0,2021.0,7.0,2021.0,0,1,0
2,6.28,30,1,2.25,0.38,846.64,7471,16.63,4.0,2021.0,4.0,2021.0,0,0,1
3,4.00,30,1,1.71,0.31,615.86,2143,17.20,4.0,2021.0,7.0,2021.0,0,0,1
4,6.18,30,1,2.25,0.38,846.64,5577,16.63,4.0,2021.0,4.0,2021.0,0,0,1


In [4]:
X = df.drop('selling_price', axis = 1)
y = df['selling_price']

In [5]:
X.shape

(53676, 14)

In [6]:
y.shape

(53676,)

In [8]:
def run_regression(pipeline, param_grid, X, y):
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)
    
    X_train = pipeline.named_steps['scaler'].fit_transform(X_train)
    
    X_test = pipeline.named_steps['scaler'].transform(X_test)
    
    grid = GridSearchCV(pipeline, param_grid, cv = 5, scoring='neg_mean_squared_error')
    
    grid.fit(X_train, y_train)
    
    print('\nBest Parameters: ',grid.best_params_)
    
    y_pred = grid.predict(X_test)
    
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = root_mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    print('Mean Absolute Error (MAE): ', mae)
    print('Mean Squared Error (MSE): ', mse)
    print('Root Mean Squared Error (RMSE): ', rmse)
    print('R-squared (R2): ', r2)

### ElasticNet Regression

In [9]:
pipeline_elasticnet = Pipeline([
    ('scaler', StandardScaler()),
    ('elasticnet', ElasticNet())
])

param_grid_elasticnet = {
    'elasticnet__alpha': [0.001, 0.01, 0.1, 1, 10, 100],
    'elasticnet__l1_ratio': [0.1, 0.5, 0.9],
    'elasticnet__max_iter': [1000]
}

run_regression(pipeline_elasticnet, param_grid_elasticnet, X, y)


Best Parameters:  {'elasticnet__alpha': 0.001, 'elasticnet__l1_ratio': 0.1, 'elasticnet__max_iter': 1000}
Mean Absolute Error (MAE):  0.42273618992622936
Mean Squared Error (MSE):  0.2970620653310992
Root Mean Squared Error (RMSE):  0.5450340038301272
R-squared (R2):  0.6942205519932709


### Gradient Boosting Regressor

In [10]:
pipeline_gb = Pipeline([
    ('scaler', StandardScaler()),
    ('gb', GradientBoostingRegressor())
])

param_grid_gb = {
    'gb__n_estimators': [10, 20],
    'gb__learning_rate': [0.01, 0.1],
    'gb__max_depth': [3, 4, 5],
    'gb__min_samples_split': [2, 5],
    'gb__min_samples_leaf': [1, 2]
}

run_regression(pipeline_gb, param_grid_gb, X, y)


Best Parameters:  {'gb__learning_rate': 0.1, 'gb__max_depth': 5, 'gb__min_samples_leaf': 1, 'gb__min_samples_split': 2, 'gb__n_estimators': 20}
Mean Absolute Error (MAE):  0.36360034679560244
Mean Squared Error (MSE):  0.22365142056339238
Root Mean Squared Error (RMSE):  0.47291798502847443
R-squared (R2):  0.7697854559464835


### Random Forest Regressor

In [11]:
pipeline_rf = Pipeline([
    ('scaler', StandardScaler()),
    ('rf', RandomForestRegressor())
])

param_grid_rf = {
    'rf__n_estimators': [10, 20],
    'rf__max_depth': [None, 5],
    'rf__min_samples_split': [2, 5],
    'rf__min_samples_leaf': [1, 2]
}

run_regression(pipeline_rf, param_grid_rf, X, y)


Best Parameters:  {'rf__max_depth': None, 'rf__min_samples_leaf': 1, 'rf__min_samples_split': 2, 'rf__n_estimators': 20}
Mean Absolute Error (MAE):  0.1615394774857096
Mean Squared Error (MSE):  0.06941485947805068
Root Mean Squared Error (RMSE):  0.2634669988405582
R-squared (R2):  0.9285481389520222


### XGBoost Regressor

In [12]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('xgb', XGBRegressor())
])

param_grid = {
    'xgb__n_estimators': [100, 200, 300], 
    'xgb__learning_rate': [0.01, 0.1, 0.2], 
    'xgb__max_depth': [3, 4, 5] 
}

run_regression(pipeline, param_grid, X, y)


Best Parameters:  {'xgb__learning_rate': 0.2, 'xgb__max_depth': 5, 'xgb__n_estimators': 300}
Mean Absolute Error (MAE):  0.18007928018309954
Mean Squared Error (MSE):  0.06920521321607989
Root Mean Squared Error (RMSE):  0.26306883740967857
R-squared (R2):  0.9287639373515608


### LightGBM 

In [13]:
pipeline = Pipeline([
    ('scaler', StandardScaler()), 
    ('lgbm', LGBMRegressor())
])

param_grid = {
    'lgbm__n_estimators': [100, 200, 300],  
    'lgbm__learning_rate': [0.01, 0.1, 0.2],
    'lgbm__max_depth': [3, 4, 5]
}

run_regression(pipeline, param_grid, X, y)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001281 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 914
[LightGBM] [Info] Number of data points in the train set: 30058, number of used features: 14
[LightGBM] [Info] Start training from score 15.024727
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001042 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 915
[LightGBM] [Info] Number of data points in the train set: 30058, number of used features: 14
[LightGBM] [Info] Start training from score 15.026697
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000963 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enoug

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001070 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 915
[LightGBM] [Info] Number of data points in the train set: 30058, number of used features: 14
[LightGBM] [Info] Start training from score 15.026697
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001134 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 913
[LightGBM] [Info] Number of data points in the train set: 30058, number of used features: 14
[LightGBM] [Info] Start training from score 15.024061
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000931 seconds

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001000 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 915
[LightGBM] [Info] Number of data points in the train set: 30058, number of used features: 14
[LightGBM] [Info] Start training from score 15.026697
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further sp

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000908 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 915
[LightGBM] [Info] Number of data points in the train set: 30059, number of used features: 14
[LightGBM] [Info] Start training from score 15.027986
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Wa

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001060 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 914
[LightGBM] [Info] Number of data points in the train set: 37573, number of used features: 14
[LightGBM] [Info] Start training from score 15.026126
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive g

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

### Conclusion

### Out of all the models, XGBoost Regressor scored the highest in all the metrics such as MAE, MSE, RMSE, R2

In [19]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('xgb', XGBRegressor())
])

param_grid = {
    'xgb__n_estimators': [300], 
    'xgb__learning_rate': [0.2], 
    'xgb__max_depth': [5]
}

run_regression(pipeline, param_grid, X, y)


Best Parameters:  {'xgb__learning_rate': 0.2, 'xgb__max_depth': 5, 'xgb__n_estimators': 300}
Mean Absolute Error (MAE):  0.18007928018309954
Mean Squared Error (MSE):  0.06920521321607989
Root Mean Squared Error (RMSE):  0.26306883740967857
R-squared (R2):  0.9287639373515608


### Saving the model

In [20]:
import joblib

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

pipeline.fit(X_train, y_train)

joblib.dump(pipeline, 'regression_model.joblib')

['regression_model_mod.joblib']

In [21]:
# Load the model from the file
loaded_model = joblib.load('regression_model.joblib')

# Use the loaded model for predictions
loaded_model.predict(X_test)

array([16.212664, 16.154123, 16.044441, ..., 13.912122, 13.081184,
       15.302719], dtype=float32)

In [17]:
# X_test.iloc[0,:]

In [18]:
# y_test